In [74]:
import os
from os import path
import numpy as np
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.models import Model
import shutil

In [10]:
if not path.isdir('val_dataset'):
    os.mkdir('val_dataset')

folders = os.listdir('dataset/')
for f in folders:
    p = path.join('val_dataset/', f)
    if not path.isdir(p):
        os.mkdir(p)

In [14]:
split = 0.9
for f in folders:
    p = path.join('dataset/', f)
    imgs = os.listdir(p)

    file_to_move = imgs[int(len(imgs) * split):]
    for img_f in file_to_move:
        src = path.join(p, img_f)
        dest = path.join('val_dataset/' + f, img_f)
        # shutil.move(src, dest)

In [75]:
# labels_dict = dict([(k, i) for i, k in enumerate(folders)])
# labels_dict

- load images from folders xtrain,  attach labels ytrain
- shuffle through zipping them
- initialize model RESNET
- compile and train
- predict

In [54]:
train_gen = image.ImageDataGenerator(rescale=1.0/255, preprocessing_function=preprocess_input)
train_generator = train_gen.flow_from_directory(
    "dataset/",
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=150,
    shuffle=True,
)

val_gen = image.ImageDataGenerator(rescale=1.0/255, preprocessing_function=preprocess_input)
val_generator = val_gen.flow_from_directory(
    "val_dataset/",
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=50,
    shuffle=True,
)

Found 19270 images belonging to 150 classes.
Found 2044 images belonging to 150 classes.


In [31]:
model = ResNet50(include_top=False, input_shape=(224, 224, 3))

94765736/94765736 [==============================] - 37s 0us/step


In [1]:
model.summary()

NameError: name 'model' is not defined

In [35]:
av1 = GlobalAveragePooling2D()(model.output)
fc1 = Dense(256, activation='relu')(av1)
d1 = Dropout((0.25))(fc1)
fc2 = Dense(150, activation='softmax')(d1)

In [37]:
model_new = Model(inputs=model.input, outputs=fc2)

In [46]:
model_new.compile(optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [43]:
model_new.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [65]:
for i in range(len(model_new.layers)):
    if i > 160:
        break
    model_new.layers[i].trainable=False
model_new.summary() 

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [70]:
hist = model_new.fit_generator(
    train_generator,
    epochs=5,
    steps_per_epoch=len(train_generator),
    validation_data=val_generator,
    validation_steps=len(val_generator)
)

C:\Users\hhars\AppData\Local\Temp\ipykernel_12696\3385218307.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(


RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.